# Common queries

Here we will collect useful queries to help new users learn by example.

In [12]:
import datajoint as dj
from pipeline import experiment, vis, preprocess, tuning

## Keys for scans with a specific type of stimulus

In [13]:
my_scans = preprocess.Sync() & vis.Monet()  # this is only a relation, no data retrieve yet
len(my_scans)

127

Get the list of primary key values for `my_scans`.

In [14]:
keys = list(my_scans.fetch.keys())
key = keys[10]  # pick an arbitrary scan from the list
key

OrderedDict([('animal_id', 7816), ('session', 1), ('scan_idx', 34)])

You may now use any one of these keys to get relevant information from other tables.

In [15]:
scan_info = experiment.Scan()*experiment.Session() & key
scan_info   # still only a query

animal_id,session,scan_idx,lens,brain_area,laser_wavelength,laser_power,filename,depth,scan_notes,site_number,software,version,scan_ts,rig,session_date,username,anesthesia,pmt_filter_set,scan_path,behavior_path,craniotomy_notes,session_notes,session_ts
7816,1,34,25x,V1,920.0,60.0,m7816A_00034,256,,9,scanimage,5.1,2016-02-11 11:38:12,2P3,2016-02-10,jake,awake,2P3 red-green A,/scratch01/Two-Photon/Jake/160210,/scratch01/WholeCell/jake/160210,,;;Emx1 Cre,2016-02-11 10:51:26


In [16]:
scan_info.fetch1()   #  now we get it in python

OrderedDict([('animal_id', 7816),
             ('session', 1),
             ('scan_idx', 34),
             ('lens', '25x'),
             ('brain_area', 'V1'),
             ('laser_wavelength', 920.0),
             ('laser_power', 60.0),
             ('filename', 'm7816A_00034'),
             ('depth', 256),
             ('scan_notes', ''),
             ('site_number', 9),
             ('software', 'scanimage'),
             ('version', '5.1'),
             ('scan_ts', datetime.datetime(2016, 2, 11, 11, 38, 12)),
             ('rig', '2P3'),
             ('session_date', datetime.date(2016, 2, 10)),
             ('username', 'jake'),
             ('anesthesia', 'awake'),
             ('pmt_filter_set', '2P3 red-green A'),
             ('scan_path', '/scratch01/Two-Photon/Jake/160210'),
             ('behavior_path', '/scratch01/WholeCell/jake/160210'),
             ('craniotomy_notes', ''),
             ('session_notes', ';;Emx1 Cre'),
             ('session_ts', datetime.datetime(2016,

## Get the stimulus info for a given scan

Let's assume you already have a key identifying a unique scan (see above). It must provide valid `animal_id`, `session`, and `scan_idx`.  You do not need to know what stimulus was played yet.

In [17]:
key

OrderedDict([('animal_id', 7816), ('session', 1), ('scan_idx', 34)])

The key identifies the scan but not the stimulus information, so you need to join the linking table `preprocess.Sync` and to limit trials to those within the scan.  The following query extracts all the relevant generic information about stimulus trials for the given scan.

In [18]:
trials = preprocess.Sync()*vis.Trial()*vis.Condition() & 'trial_idx between first_trial and last_trial' & key  

You can now join this relation with its specific type of stimulus to get the stimulus-specific information.  Beware that within a single scan multiple types of stimuli may be played. For those stimulus conditions that have a lookup, join the lookup too.

In [19]:
monet_trials = trials*vis.Monet()*vis.MonetLookup()
movie_trials = trials*vis.Movie()*vis.MovieClipCond()
trippy_trials = trials*vis.Trippy()   # no lookup

## Get the fluorophores used in a given scan

Say you already have the key of a scan or a session.

In [20]:
key = dict(animal_id=8158, session=1)

In [21]:
fluorophores = experiment.Session.Fluorophore() & key
fluorophores

animal_id,session,fluorophore,notes
8158,1,Twitch2B,


## Get synced scans with a given fluorophore

In [23]:
twitchScans = preprocess.Sync() & (experiment.Session.Fluorophore() & dict(fluorophore="Twitch2B"))